In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install transformers

In [ ]:
!nvidia-smi

Thu Sep 22 05:46:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip3 install ipython-autotime
%load_ext autotime

time: 661 µs (started: 2022-09-22 05:46:52 +00:00)


In [ ]:
import re
import pandas as pd
import numpy as np
import json
import torch
import contextlib, unicodedata, sys

In [ ]:
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

time: 1.34 ms (started: 2022-06-02 17:59:28 +00:00)


In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

time: 1.64 ms (started: 2022-06-02 17:59:28 +00:00)


#### Data Input

In [ ]:
'''
keys:
    title: wiki title pages's title
    event_time: event occur time
    title_update_time: wiki title page update time (after event occur)
    event_link
    event_wcep_id: id that is same as WCEP dateset
    title_update_link: wiki title page's link after update (after event occur)
    update_bs4: wiki title page's html after update (after event occur)
    title_non_update_link: wiki title page link before update (befor event occur)
    title_non_update_time: wiki title page's time before event occur 
    non_update_bs4: wiki title page's html before update (before event occur)

    update_first_paragraph: wiki title page's first paragraph (after event occur)
    # update_all_paragraph: wiki title page's all paragraphs (after event occur)
    non_update_first_paragraph: wiki title page's first paragraph (before event occur)
    non_update_all_paragraph: wiki title page's all paragraph (before event occur)
    # wiki_summary: wiki current event portal summary (from WCEP dataset)
    # source_news_first: wiki portal news first paragraph
    # source_news_full: wiki portal news all paragraph

'''

# sources
# all paragraphs + wiki_summary (non_update_all_paragraph + wiki_summary)
# all paragraphs + wiki portal news first (non_update_all_paragraph + source_news_first)
# all paragraphs + wiki portal news full (non_update_all_paragraph + source_news_full)


# target
# all paragraph (update_all_paragraph)

# new pairs
# source_news_first/wiki_summary \ updated_all_paragraph

In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom

/content/drive/MyDrive/MDS/wcep_custom


In [ ]:
'title', 
'event_time', 
'title_update_time', # 
'event_link', 
'title_update_link', 
# 'update_bs4', 
'title_non_update_link', 
'title_non_update_time', 
# 'non_update_bs4', 
'article_title', # article title from 'event link'
'source_news_first', #
'source_news_full', #
# 'article_record', 
'update_all_paragraph_with_references', 
'update_first_paragraph', #
'update_all_paragraph', # 
'non_update_all_paragraph_with_references', 
'non_update_all_paragraph', #
'non_update_first_paragraph', # 
'update_wiki_summary', # before content box
'non_update_wiki_summary', # before content box
'wiki_portal_summary', # from wcep (meta info)
'citation_position', 
'event_wcep_id'

#### baseline 1: non_update_all_paragraph + wiki_summary

In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom
import json
clusters = []
with open("title_page_whole_test.txt", 'r') as fread:
    for line in fread.readlines():
        clusters.append(json.loads(line.strip()))

/content/drive/MyDrive/MDS/wcep_custom


In [ ]:
gen_clusters = []
with open('bs1_test/filtered_non/gen_summ_tuned/gen_summ_tuned.test.tgt', 'r') as fread:
    for line in fread.readlines():
        gen_clusters.append(line.strip())

In [ ]:
whole_clusters = []
with open("title_page_whole_test.txt", 'r') as fread:
    for line in fread.readlines():
        whole_clusters.append(line.strip()))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
clusters[0].keys()

dict_keys(['title', 'event_time', 'title_update_time', 'event_link', 'title_update_link', 'update_bs4', 'title_non_update_link', 'title_non_update_time', 'non_update_bs4', 'article_title', 'source_news_first', 'source_news_full', 'article_record', 'update_all_paragraph_with_references', 'update_first_paragraph', 'update_all_paragraph', 'non_update_all_paragraph_with_references', 'non_update_all_paragraph', 'non_update_first_paragraph', 'update_wiki_summary', 'non_update_wiki_summary', 'wiki_portal_summary', 'citation_position', 'event_wcep_id'])

time: 3.83 ms (started: 2022-09-22 05:47:17 +00:00)


In [ ]:
clusters[0]['non_update_first_paragraph']

'The 2020 Summer Olympics (Japanese: 2020年夏季オリンピック, Hepburn: Nisen Nijū-nen Kaki Orinpikku), officially the Games of the XXXII Olympiad (第三十二回オリンピック競技大会, Dai Sanjūni-kai Orinpikku Kyōgi Taikai) and branded as Tokyo 2020 (東京2020), is an ongoing international multi-sport event being held from 23 July to 8\xa0August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July.'

time: 3.71 ms (started: 2022-09-22 05:47:17 +00:00)


In [ ]:
clusters[0]['wiki_portal_summary']

'Burkinabé athlete Hugues Fabrice Zango gets third place in the men’s triple jump event, winning Burkina Faso its first Olympic medal. '

time: 2.28 ms (started: 2022-09-22 05:47:17 +00:00)


In [ ]:
clusters[0]['title_non_update_link']

'https://en.wikipedia.org//w/index.php?title=2020_Summer_Olympics&oldid=1037249519'

time: 2.77 ms (started: 2022-09-22 05:47:17 +00:00)


In [ ]:
clusters[0]['title_update_link']

'https://en.wikipedia.org/w/index.php?title=2020_Summer_Olympics&oldid=1037249733'

time: 4.34 ms (started: 2022-09-22 05:47:17 +00:00)


In [ ]:
clusters[0]['event_link']

'https://punchng.com/breaking-hugues-zango-wins-burkina-fasos-first-ever-olympic-medal/'

time: 4.46 ms (started: 2022-09-22 05:47:54 +00:00)


In [ ]:
clusters[0].keys()

dict_keys(['title', 'event_time', 'title_update_time', 'event_link', 'title_update_link', 'update_bs4', 'title_non_update_link', 'title_non_update_time', 'non_update_bs4', 'article_title', 'source_news_first', 'source_news_full', 'article_record', 'update_all_paragraph_with_references', 'update_first_paragraph', 'update_all_paragraph', 'non_update_all_paragraph_with_references', 'non_update_all_paragraph', 'non_update_first_paragraph', 'update_wiki_summary', 'non_update_wiki_summary', 'wiki_portal_summary', 'citation_position', 'event_wcep_id'])

time: 2.81 ms (started: 2022-05-23 01:33:43 +00:00)


In [ ]:
# create idx list
error_idx = []
all_idx = [i for i in range(len(clusters))]
len_non_paragraph_len = []
len_all = []

for idx in range(len(clusters)):
    # src: non_update_all_paragraph + wiki_summary
    try:
        if bool(len(clusters[idx]['non_update_all_paragraph'].split('. ')) == 1):
            error_idx.append(idx)
    except:
        error_idx.append(idx)
idx_list = Diff(all_idx, error_idx)

time: 158 ms (started: 2022-05-23 01:33:43 +00:00)


In [ ]:
err_idx = []
for idx in idx_list:
    if len(clusters[idx]['update_first_paragraph'].split())<=10:
        err_idx.append(idx)
idx_fur_idx = Diff(idx_list, err_idx)

time: 36.4 ms (started: 2022-05-23 01:33:43 +00:00)


In [ ]:
print(len(clusters), len(idx_list), len(idx_fur_idx))

1906 1868 1808
time: 892 µs (started: 2022-05-23 01:33:43 +00:00)


In [ ]:
# 
wiki_portal_summaries = [clusters[idx]['wiki_portal_summary'].strip() for idx in idx_fur_idx]
wiki_portal_summaries_all = [clusters[idx]['wiki_portal_summary'].strip() for idx in range(len(clusters))]
print(len(wiki_portal_summaries_all))   
print(len(wiki_portal_summaries)) # the index is followed after idx_fur_idx
print(len(set(wiki_portal_summaries)))

1906
1808
1633
time: 5.67 ms (started: 2022-05-23 01:33:43 +00:00)


In [ ]:
# check the first seen cluster index in dataset
seen = []
duplicated_ord = []
kept_ord = []
duplicated_rec = {}
for idx in range(len(wiki_portal_summaries)):
    summary = wiki_portal_summaries[idx]
    if summary not in seen:
        kept_ord.append(idx)
        seen.append(summary)
# store real indices for duplicated clusters
kept_idx = [idx_fur_idx[ord] for ord in kept_ord]
# using the kept idx to find its duplicated ids
duplicated_ids = []
for idx in kept_idx:
    summary = clusters[idx]['wiki_portal_summary'].strip()
    duplicated_id = []
    for id in idx_fur_idx:
        if clusters[id]['wiki_portal_summary'].strip()==summary:
            duplicated_id.append(id)
    duplicated_id.remove(idx)
    duplicated_ids.append(duplicated_id)
rec_idx = kept_idx
rec_val = duplicated_ids
rec = dict(zip(rec_idx, rec_val))

time: 1.06 s (started: 2022-05-23 01:33:43 +00:00)


In [ ]:
pairs, empty_lst_idx = [], []
# pairs: collected pairs of duplicated cluster indices
# empty_lst_idx: non duplicated cluster indices
for idx in list(rec.keys()):
    if len(rec[idx])!=0:
        pair = [idx]
        pair.extend(rec[idx])
        pair = sorted(list(set(pair)))
        pairs.append(pair)
    if len(rec[idx])==0:
        empty_lst_idx.append(idx)
print(len(pairs))
print(len(empty_lst_idx))

144
1489
time: 5.55 ms (started: 2022-05-23 01:33:44 +00:00)


In [ ]:
# keep the latest from duplicated clusters
import datetime
months_dict_idx = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
months_dict_vals = [i for i in range(1, 13)]
months_dict = dict(zip(months_dict_idx, months_dict_vals))
the_latest = []
for pair in pairs:
    datetime_lst = []
    for idx in pair:
        time_lst = clusters[idx]['title_update_time'].split()
        year = int(time_lst[-1])
        month = months_dict[time_lst[-2]]
        date = int(time_lst[-3])
        hr = int(time_lst[0].split(':')[0])
        min = int(time_lst[0].split(':')[1].rstrip(','))
        datetime_lst.append(datetime.datetime(year, month, date, hr, min, 0))
    the_latest.append(pair[datetime_lst.index(max(datetime_lst))])
# merge
rmved_idx = sorted(empty_lst_idx+the_latest)

time: 10.1 ms (started: 2022-05-23 01:33:44 +00:00)


In [ ]:
len(rmved_idx)

1633

time: 2.67 ms (started: 2022-05-23 01:33:44 +00:00)


In [ ]:
# filter by language of summ==English
rmved_idx_en, idx_fur_idx_en = [], []
for idx in rmved_idx:
    summ_str = clusters[idx]['wiki_portal_summary']
    if isEnglish(summ_str):
        rmved_idx_en.append(idx)
for idx in idx_fur_idx:
    summ_str = clusters[idx]['wiki_portal_summary']
    if isEnglish(summ_str):
        idx_fur_idx_en.append(idx)

time: 10.1 ms (started: 2022-05-23 01:33:44 +00:00)


In [ ]:
print(len(rmved_idx_en), len(idx_fur_idx_en))

1454 1602
time: 949 µs (started: 2022-05-23 01:33:44 +00:00)


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs1_train/filtered
%mkdir without_update_src
%cd without_update_src

/content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered
mkdir: cannot create directory ‘without_update_src’: File exists
/content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered/without_update_src
time: 433 ms (started: 2022-05-15 15:20:18 +00:00)


In [ ]:
clusters[0].keys()

dict_keys(['title', 'event_time', 'title_update_time', 'event_link', 'title_update_link', 'update_bs4', 'title_non_update_link', 'title_non_update_time', 'non_update_bs4', 'article_title', 'source_news_first', 'source_news_full', 'article_record', 'update_all_paragraph_with_references', 'update_first_paragraph', 'update_all_paragraph', 'non_update_all_paragraph_with_references', 'non_update_all_paragraph', 'non_update_first_paragraph', 'update_wiki_summary', 'non_update_wiki_summary', 'wiki_portal_summary', 'citation_position', 'event_wcep_id'])

time: 2.76 ms (started: 2022-05-20 18:12:24 +00:00)


In [ ]:
src_data_pre = []
with open('bef.test.txt.src', 'w') as fwrite:
    for idx in idx_fur_idx_en:
        bef_rec = []
        # wiki_summary = [clusters[idx]['wiki_portal_summary']]
        non_update_all_paragraph = clusters[idx]['non_update_all_paragraph'].split('\n')
        bef_rec.extend(non_update_all_paragraph)
        # bef_rec.extend(wiki_summary)
        s = '.\c\c'
        bef_rec_str = s.join(bef_rec)
        with contextlib.redirect_stdout(fwrite):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})

time: 171 ms (started: 2022-05-15 06:34:55 +00:00)


In [ ]:
'''
src_data_pre = []
with open('befold.test.txt.src', 'w') as fwrite:
    for idx in idx_fur_idx:
        bef_rec = []
        # wiki_summary = [clusters[idx]['wiki_summary']]
        non_update_all_paragraph = clusters[idx]['non_update_all_paragraph'].split('\n')
        bef_rec.extend(non_update_all_paragraph)
        # bef_rec.extend(wiki_summary)
        s = '.\c\c'
        bef_rec_str = s.join(bef_rec)
        with contextlib.redirect_stdout(fwrite):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})
'''

"\nsrc_data_pre = []\nwith open('befold.test.txt.src', 'w') as fwrite:\n    for idx in idx_fur_idx:\n        bef_rec = []\n        # wiki_summary = [clusters[idx]['wiki_summary']]\n        non_update_all_paragraph = clusters[idx]['non_update_all_paragraph'].split('\n')\n        bef_rec.extend(non_update_all_paragraph)\n        # bef_rec.extend(wiki_summary)\n        s = '.\\c\\c'\n        bef_rec_str = s.join(bef_rec)\n        with contextlib.redirect_stdout(fwrite):\n            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})\n"

time: 3.13 ms (started: 2022-05-15 06:34:55 +00:00)


In [ ]:
import re
check_src = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        line = re.sub("[\"\'\{\}]", "", line)
        line = line.strip()
        check_src.append(line)
print(len(check_src))

with open('bef.test.txt.src', 'w') as fwrite:
    for line in check_src:
        line = line.replace('\\\\c\\\\c', '\\c\\c')
        # for without validation
        # line = line.replace('\\c\\c', ' `` ')
        fwrite.write(line+'\n')

177
time: 98.4 ms (started: 2022-05-15 06:34:55 +00:00)


In [ ]:
'''
import re
check_src = []
with open('befold.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        line = re.sub("[\"\'\{\}]", "", line)
        line = line.strip()
        check_src.append(line)
print(len(check_src))

with open('befold.test.txt.src', 'w') as fwrite:
    for line in check_src:
        line = line.replace('\\\\c\\\\c', '\\c\\c')
        # for without validation
        # line = line.replace('\\c\\c', ' `` ')
        fwrite.write(line+'\n')
'''

'\nimport re\ncheck_src = []\nwith open(\'befold.test.txt.src\', \'r\') as fread:\n    for line in fread.readlines():\n        line = re.sub("["\'\\{\\}]", "", line)\n        line = line.strip()\n        check_src.append(line)\nprint(len(check_src))\n\nwith open(\'befold.test.txt.src\', \'w\') as fwrite:\n    for line in check_src:\n        line = line.replace(\'\\\\c\\\\c\', \'\\c\\c\')\n        # for without validation\n        # line = line.replace(\'\\c\\c\', \' `` \')\n        fwrite.write(line+\'\n\')\n'

time: 2.87 ms (started: 2022-05-15 06:34:56 +00:00)


In [ ]:
check_src_fur = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        check_src_fur.append(line)
print(len(check_src_fur))

177
time: 17.4 ms (started: 2022-05-15 06:34:56 +00:00)


In [ ]:
'''
check_src_fur = []
with open('befold.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        check_src_fur.append(line)
print(len(check_src_fur))
'''

"\ncheck_src_fur = []\nwith open('befold.test.txt.src', 'r') as fread:\n    for line in fread.readlines():\n        check_src_fur.append(line)\nprint(len(check_src_fur))\n"

time: 3.02 ms (started: 2022-05-15 06:34:56 +00:00)


In [ ]:
check_src_fur[:5]

['On October 8, 2020, the U.S. Federal Bureau of Investigation (FBI) announced the arrests of 13 men suspected of orchestrating a domestic terror plot to kidnap Gretchen Whitmer, the Governor of Michigan, and otherwise violently overthrow the state government. The suspects were tied to a paramilitary militia group that called themselves the Wolverine Watchmen, which was founded by two of the suspects. Six of the suspects were charged in federal court, while the other seven were charged with state crimes. A week later, a fourteenth suspect was arrested and charged in state court..\\c\\cPreceding the 2020 United States elections, law enforcement officials, members of Congress, and groups tracking extremism in the country have warned about the increasing threat potential from militia and far-right groups. The Anti-Defamation League identified Michigan as a state where the modern militia movement found its roots and where a number of militia groups remain active. However, Jon Lewis, a rese

time: 5.65 ms (started: 2022-05-15 06:34:57 +00:00)


In [ ]:
clusters[0].keys()

dict_keys(['title', 'event_time', 'title_update_time', 'event_link', 'title_update_link', 'update_bs4', 'title_non_update_link', 'title_non_update_time', 'non_update_bs4', 'article_title', 'source_news_first', 'source_news_full', 'article_record', 'update_all_paragraph_with_references', 'update_first_paragraph', 'update_all_paragraph', 'non_update_all_paragraph_with_references', 'non_update_all_paragraph', 'non_update_first_paragraph', 'update_wiki_summary', 'non_update_wiki_summary', 'wiki_portal_summary', 'citation_position', 'event_wcep_id'])

time: 3.37 ms (started: 2022-05-19 07:14:34 +00:00)


In [ ]:
# tgt: udpate_all_paragraph
%cd /content/drive/MyDrive/MDS/wcep_custom/bs1_test/filtered_non
tgt_clusters = []
check_output = []

for idx in rmved_idx_en:
    cleaned_article = re.sub(r'\[.*?\]', "", clusters[idx]['update_all_paragraph'])
    output_article = str(cleaned_article).rstrip('\n')
    output_article = str(output_article).rstrip()
    tgt_clusters.append(output_article.replace('\n', '\c'))

/content/drive/MyDrive/MDS/wcep_custom/bs1_test/filtered_non
time: 130 ms (started: 2022-05-23 01:37:17 +00:00)


In [ ]:
with open('test.txt.tgt', 'w') as fwrite:
    for line in tgt_clusters:
        fwrite.write('%s\n' % line)

with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        check_output.append(line)

time: 670 ms (started: 2022-05-20 18:21:20 +00:00)


In [ ]:
check_output[2]

'Jean-Pierre Bemba (born 4 November 1962) was one of four vice-presidents in the transitional government of the Democratic Republic of the Congo from 2003 to 2006. He was sworn in on 17 July 2003. Bemba also leads the Movement for the Liberation of Congo (MLC), a rebel group turned political party. He received the second highest number of votes in the 2006 presidential election. In January 2007 he was elected to the Senate.\\cBemba is one of the richest men in the Congo, with an estimated fortune of several hundred million dollars.  His businesses have included portable radios, aviation and private television stations. His father was a businessman who was successful under former Zairian dictator Mobutu Sese Seko, and one of his sisters is married to Mobutu\'s son Nzanga, who was also a candidate in the 2006 presidential elections.\\cIn 2002, President Ange-Félix Patassé of the Central African Republic invited the MLC to come to his country and put down a coup attempt; human rights acti

time: 4.05 ms (started: 2022-05-20 18:21:20 +00:00)


In [ ]:
intermediate = []

with open("output_intermediate.txt", "w") as o:
    for idx in range(len(tgt_clusters)):
        check_txt = check_output[idx]
        with contextlib.redirect_stdout(o):
            print({unicodedata.normalize('NFKD', check_txt).encode('utf-8', 'ignore').decode('utf-8')})

intermediate = []
with open('output_intermediate.txt', 'r') as fwrite:
    for line in fwrite.readlines():
        intermediate.append(line)
    for id in range(len(intermediate)):
        intermediate[id] = intermediate[id].replace("{b'", "").replace("\\n'}\n", "").replace('{b"', '').replace('\\n"}\n', '')
        re.sub("[\"\'\{\}]", "", line)
        intermediate[id] = intermediate[id].replace('.. ', '. ')

time: 1.59 s (started: 2022-05-20 18:21:20 +00:00)


In [ ]:
with open('test.txt.tgt', 'w') as fwrite:
    for line in intermediate:
        fwrite.writelines(line+'\n')

time: 4.06 s (started: 2022-05-20 18:21:22 +00:00)


In [ ]:
check_tgt = []
with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        line = re.sub("[\"\'\{\}]", "", line)
        line = line.strip()
        line = line.replace('\\\\c', '\n')
        check_tgt.append(line)

time: 450 ms (started: 2022-05-20 18:21:26 +00:00)


In [ ]:
len(check_tgt)

1454

time: 2.71 ms (started: 2022-05-20 18:21:26 +00:00)


In [ ]:
%cd without_update_src
%rm test.txt.tgt
%cd ..

/content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered/without_update_src
/content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered
time: 240 ms (started: 2022-05-20 18:21:26 +00:00)


In [ ]:
len_rec = []
for tgt in check_tgt:
    len_rec.append(len(tgt.split()))
print(np.min(len_rec))
print(np.max(len_rec))
print(np.mean(len_rec))
print(np.median(len_rec))

15
27130
3673.5110041265475
2055.5
time: 280 ms (started: 2022-05-20 18:21:27 +00:00)


In [ ]:
check_tgt[3]

'The Gaza–Israel conflict is a part of the localized Israeli–Palestinian conflict, but is also a scene of power struggle between regional powers including Egypt, Iran and Turkey together with Qatar, supporting different sides of the conflict in light of the regional standoff between Iran and Saudi Arabia on one hand and between Qatar and Saudi Arabia on the other, as well as crisis in Egyptian-Turkish relations.\nThe conflict originated with the election of the Islamist political party Hamas in 2005 and 2006 in the Gaza Strip and escalated with the split of the Palestinian Authority Palestinian government into the Fatah government in the West Bank  and the Hamas government in Gaza and the following violent ousting of Fatah after Fatah lost the election to Hamas. Palestinian rocket attacks on Israel, Israeli airstrikes on Gaza, and the joint Egyptian-Israeli blockade of Gaza have exacerbated the conflict. The international community considers indiscriminate attacks on civilians and civi

time: 2.76 ms (started: 2022-05-19 07:58:02 +00:00)


In [ ]:
clusters[4]['update_all_paragraph']

'On April 15, 2021, a mass shooting occurred at a FedEx Ground facility in Indianapolis, Indiana, United States. Nine people were killed, including the gunman, 19-year-old former employee Brandon Scott Hole, who committed suicide. Seven others were injured, including four by gunfire. Hole\'s motive remains unknown.\nThe FedEx facility where the shooting occurred is located in the southwestern part of the city, near Indianapolis International Airport. It is the company\'s second-busiest hub, with nearly 4,000 employees in total. The building is equipped with metal detectors and security turnstiles at its entrance requiring the employees to scan their FedEx badges. There were at least 100 people in the facility at the time of the shooting, and many of them were changing shifts or on dinner breaks.\nAccording to Indianapolis Deputy Police Chief Craig McCartt, the gunman entered the facility in a vehicle and, after exiting unprovoked, immediately began firing. Afterwards, the gunman entere

time: 3.38 ms (started: 2022-05-19 07:58:03 +00:00)


##### Remove duplicated sentences

In [ ]:
train_pt = torch.load('/content/drive/MyDrive/MDS_PRIMER/primer/dataset/wcep-10/wceptrain.pt')
# train_pt = torch.load('/content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered/wceptest_train.pt')

time: 3.26 s (started: 2022-05-14 14:31:23 +00:00)


In [ ]:
%pwd

'/content/drive/MyDrive/MDS/wcep_custom/bs2_valid/filtered'

time: 3.35 ms (started: 2022-05-14 15:29:06 +00:00)


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs1_test/filtered_non

/content/drive/MyDrive/MDS/wcep_custom/bs1_test/filtered_non
time: 10.4 ms (started: 2022-06-02 18:00:02 +00:00)


In [ ]:
%ls

aft_model_txt.src       gen_summ/                test.txt.tgt
aft_test_rouge_txt.src  gen_summ_tuned/          wceptest_test.pt
bef.test.txt.src        output_intermediate.txt  without_update_src/
time: 114 ms (started: 2022-06-02 18:02:39 +00:00)


In [ ]:
src_clusters = []
with open("bef.test.txt.src", 'r') as fread:
    for line in fread.readlines():
        src_clusters.append(line)

time: 1.56 s (started: 2022-06-02 18:00:13 +00:00)


In [ ]:
### This for further filtering for testing set and validation set (removed duplicated sentences appear in training set)
src_clusters = []
with open("aft_model_txt.src", 'r') as fread:
    for line in fread.readlines():
        src_clusters.append(line)

In [ ]:
# create sentences for finding duplicated docs - training set
concat_sents_cluster = []
for idx in range(len(train_pt)):
    sents = []
    for id in range(len(train_pt[idx]['document'])):
        sent = train_pt[idx]['document'][id].split('\n\n')
        sents.extend(sent)
    s = '. '
    concat_sents = s.join(sents)
    concat_sents_cluster.append(concat_sents)

time: 403 ms (started: 2022-05-14 14:31:49 +00:00)


In [ ]:
#sentences before filtered
total_len = 0
total_par = 0
for idx in range(len(src_clusters)):
    for id in range(len(src_clusters[idx].split('.\\c\\c'))):
        sents = src_clusters[idx].split('.\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
        total_par+=1
print(total_len)

36696
time: 913 ms (started: 2022-05-23 01:37:39 +00:00)


In [ ]:
concat_custom = []
for idx in range(len(src_clusters)):
    pars_per = src_clusters[idx].split('.\\c\\c')
    concat_custom.append(pars_per)

time: 5.84 ms (started: 2022-05-14 14:32:37 +00:00)


In [ ]:
train_pre_dup = []
test_pre_dup = concat_custom.copy()

for idx in range(len(concat_sents_cluster)):
    ele = concat_sents_cluster[idx].split('. ')
    train_pre_dup.append(ele)

time: 831 ms (started: 2022-05-14 14:32:39 +00:00)


In [ ]:
all_train_lst = []
num_sent = 0

for id in range(len(train_pre_dup)):
    for i in range(len(train_pre_dup[id])):
        all_train_lst.append(train_pre_dup[id][i])

time: 346 ms (started: 2022-05-14 14:32:41 +00:00)


In [ ]:
# assert(len(idx_fur_idx)==len(test_pre_dup))
assert len(idx_fur_idx_en)== len(test_pre_dup)

time: 854 µs (started: 2022-05-14 14:32:42 +00:00)


In [ ]:
test_pre_dup[0]

['Hugo Rafael Chávez Frías (IPA: [\\uɰo rafa\\el \\tʃaβes \\fɾias]) (born July 28, 1954) is the 53rd and current President of Venezuela. As the leader of the Bolivarian Revolution, Chávez promotes his vision of democratic socialism, Latin American integration, and anti-imperialism.  He is also an ardent critic of neoliberal globalization and  US foreign policy.',
 'A career military officer, Chávez founded the leftist Fifth Republic Movement after orchestrating a failed 1992 coup d\\état against former president Carlos Andrés Pérez.  Chávez was elected President in 1998 on promises of aiding Venezuela\\s poor majority, and was reelected in 2000. Domestically, Chávez has launched massive Bolivarian Missions, whose goals are to combat disease, illiteracy, malnutrition, poverty, and other social ills. Abroad, Chávez has acted against the Washington Consensus by supporting alternative models of economic development, and has advocated cooperation among the world\\s poor nations, e

time: 2.88 ms (started: 2022-05-14 14:32:45 +00:00)


In [ ]:
aft_processed = []
duplicated_idx = []
for idx in range(len(test_pre_dup)):
    # splitted = [test_pre_dup[idx][id].split('. ') for id in test_pre_dup]
    processed_pars = []
    for id in range(len(test_pre_dup[idx])):
        sents_in_par = test_pre_dup[idx][id].split('. ')
        dict_idx = sents_in_par
        dict_val = [0 for _ in range(len(dict_idx))]
        dict_cus = dict(zip(dict_idx, dict_val))
        for q in all_train_lst:
            try:
                if dict_cus[q]==0: 
                    sents_in_par.remove(q)
                    duplicated_idx.append(idx)
            except:
                pass
        s = '. '
        processed_par = s.join(sents_in_par)
        processed_pars.append(processed_par)
    aft_processed.append(processed_pars)
duplicated_idx_setlst = list(set(duplicated_idx))
# 'aft_model_txt.src' 'aft_model_sec_txt.src'
with open('aft_model_txt.src', 'w') as fwrite:
    for idx in range(len(aft_processed)):
        src_data = aft_processed[idx]
        s = r'.\c\c'
        src_txt = s.join(src_data)
        fwrite.write(src_txt)
# with open('duplicated_idx.txt', 'w') as fwrite:
#     for idx in duplicated_idx_setlst:
#         fwrite.write(idx+'\n')

In [ ]:
%ls .

aft_model_txt.src       gen_summ/                wceptest_test.pt
aft_test_rouge_txt.src  output_intermediate.txt  without_update_src/
bef.test.txt.src        test.txt.tgt
time: 231 ms (started: 2022-05-23 01:39:01 +00:00)


In [ ]:
check_out = []
with open('aft_test_rouge_txt.src', 'r') as f:
# with open('bef.test.txt.src', 'r') as f:
    for line in f.readlines():
        check_out.append(line)

time: 435 ms (started: 2022-06-02 18:01:14 +00:00)


In [ ]:
len(check_out)

192

time: 3.02 ms (started: 2022-05-14 04:48:49 +00:00)


In [ ]:
# new_fil_id = []
# for id in idx_fur_idx:
#     new_fil_id.append(idx_list.index(id))
# with open('aftold_test.txt.src', 'w') as f:
#     for id in new_fil_id:
#         input = check_out[id]
#         f.write(input)

time: 223 ms (started: 2022-04-21 08:21:48 +00:00)


In [ ]:
check_out[:5]

['On October 8, 2020, the U.S. Federal Bureau of Investigation (FBI) announced the arrests of 13 men suspected of orchestrating a domestic terror plot to kidnap Gretchen Whitmer, the Governor of Michigan, and otherwise violently overthrow the state government. The suspects were tied to a paramilitary militia group that called themselves the Wolverine Watchmen, which was founded by two of the suspects. Six of the suspects were charged in federal court, while the other seven were charged with state crimes. A week later, a fourteenth suspect was arrested and charged in state court..\\c\\cPreceding the 2020 United States elections, law enforcement officials, members of Congress, and groups tracking extremism in the country have warned about the increasing threat potential from militia and far-right groups. The Anti-Defamation League identified Michigan as a state where the modern militia movement found its roots and where a number of militia groups remain active. However, Jon Lewis, a rese

time: 5.4 ms (started: 2022-05-14 04:48:51 +00:00)


In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('\\c\\c', ' `` ')
    modified_out.append(line)
with open('aft_test_rouge_txt.src', 'w') as fwrite:
    for line in modified_out:
        fwrite.write(line)

time: 35.5 ms (started: 2022-05-14 04:48:52 +00:00)


In [ ]:
read_check = []
with open('aft_test_rouge_txt.src', 'r') as fread:
    for line in fread.readlines():
        read_check.append(line)
print(len(read_check))

192
time: 19.8 ms (started: 2022-05-14 04:48:53 +00:00)


In [ ]:
check_out[:5]

['On October 8, 2020, the U.S. Federal Bureau of Investigation (FBI) announced the arrests of 13 men suspected of orchestrating a domestic terror plot to kidnap Gretchen Whitmer, the Governor of Michigan, and otherwise violently overthrow the state government. The suspects were tied to a paramilitary militia group that called themselves the Wolverine Watchmen, which was founded by two of the suspects. Six of the suspects were charged in federal court, while the other seven were charged with state crimes. A week later, a fourteenth suspect was arrested and charged in state court..\\c\\cPreceding the 2020 United States elections, law enforcement officials, members of Congress, and groups tracking extremism in the country have warned about the increasing threat potential from militia and far-right groups. The Anti-Defamation League identified Michigan as a state where the modern militia movement found its roots and where a number of militia groups remain active. However, Jon Lewis, a rese

time: 78.7 ms (started: 2022-05-14 04:48:54 +00:00)


In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('.\\c\\c', '\c\c')
    modified_out.append(line)
with open('aft_model_txt.src', 'w') as fwrite:
    for line in modified_out:
        line_encoded = unicodedata.normalize('NFKD', line).encode('utf-8', 'ignore').decode('utf-8')
        # line_encoded = line.encode('unicode_escape')
        # line_encoded = line.encode('ignore')
        fwrite.write(str(line_encoded))

192
time: 93.5 ms (started: 2022-05-14 04:48:56 +00:00)


In [ ]:
read_check_model = []
with open('aft_model_txt.src', 'r') as fread:
    for line in fread.readlines():
        # read_check_model.append(line.replace('.\\c\\c', '\c\c'))
        read_check_model.append(line.strip())
print(len(read_check_model))

201
time: 430 ms (started: 2022-06-02 18:00:47 +00:00)


In [ ]:
read_check_model[:5]

['On October 8, 2020, the U.S. Federal Bureau of Investigation (FBI) announced the arrests of 13 men suspected of orchestrating a domestic terror plot to kidnap Gretchen Whitmer, the Governor of Michigan, and otherwise violently overthrow the state government. The suspects were tied to a paramilitary militia group that called themselves the Wolverine Watchmen, which was founded by two of the suspects. Six of the suspects were charged in federal court, while the other seven were charged with state crimes. A week later, a fourteenth suspect was arrested and charged in state court.\\c\\cPreceding the 2020 United States elections, law enforcement officials, members of Congress, and groups tracking extremism in the country have warned about the increasing threat potential from militia and far-right groups. The Anti-Defamation League identified Michigan as a state where the modern militia movement found its roots and where a number of militia groups remain active. However, Jon Lewis, a resea

time: 6 ms (started: 2022-05-14 04:48:56 +00:00)


In [ ]:
# prepare truncated version
import nltk
nltk.download('punkt')
mult = []   
for idx in range(len(read_check_model)):
    length = 0
    sents = []
    for sent in nltk.sent_tokenize(read_check_model[idx]):
        length += len(nltk.word_tokenize(sent))
        if length < 4096:
            sents.append(sent)
    single = ' '.join(sents)
    mult.append(single)
    
with open('afttrunc_test_model.txt.src', 'w') as fwrite:
    for line in mult:
        fwrite.write(line.strip()+'\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
time: 8.38 s (started: 2022-04-21 11:40:24 +00:00)


In [ ]:
with open('test.txt.tgt', 'r') as fread:
    check_tgt = [line.strip() for line in fread.readlines()]
print(len(check_tgt))

177
time: 2.84 ms (started: 2022-05-12 08:11:06 +00:00)


In [ ]:
len_check_tgt = [len(nltk.word_tokenize(check_tgt[idx])) for idx in range(len(check_tgt))]

time: 122 ms (started: 2022-04-21 11:40:43 +00:00)


In [ ]:
np.max(len_check_tgt)

274

time: 3.88 ms (started: 2022-04-21 11:40:46 +00:00)


In [ ]:
read_check_model[:10]

['The 2020 Summer Olympics (Japanese: 2020年夏季オリンピック, Hepburn: Nisen Nijū-nen Kaki Orinpikku), officially the Games of the XXXII Olympiad (第三十二回オリンピック競技大会, Dai Sanjūni-kai Orinpikku Kyōgi Taikai) and branded as Tokyo 2020 (東京2020), is an ongoing international multi-sport event being held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July\\c\\cTokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013. Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 in March 2020 as a result of the COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retains the Tokyo 2020 name for marketing and branding purposes. It is being held largely behind closed doors with no public spectators permitted due to the declaration of a state of emergency. The Summer

time: 5.41 ms (started: 2022-04-20 02:44:22 +00:00)


In [ ]:
cnt_par = []
for idx in range(len(check_out)):
    cnt_single_par = len(check_out[idx].split('\\c\\c'))
    cnt_par.append(cnt_single_par)
print('#paragraphs in cluster')
print(np.min(cnt_par))
print(np.max(cnt_par))
print(np.mean(cnt_par))
print(np.median(cnt_par))

cnt_sents = []
for idx in range(len(check_out)):
    cnt_single = 0
    for id in range(len(check_out[idx].split('\\c\\c'))):
        single_par = check_out[idx].split('\\c\\c')[id]
        cnt_single+=len(single_par.split('. '))
    cnt_sents.append(cnt_single)
print('#sentences in paragraph')
print(np.min(cnt_sents))
print(np.max(cnt_sents))
print(np.mean(cnt_sents))
print(np.median(cnt_sents))

cnt_single_sent = []
for idx in range(len(check_out)):
    for id in range(len(check_out[idx].split('\\c\\c'))):
        single_par = check_out[idx].split('\\c\\c')[id]
        cnt_sent_in_par = len(single_par.split('. '))
        cnt_single_sent.append(cnt_sent_in_par)


#paragraphs in cluster
2
515
78.5625
40.0
#sentences in paragraph
4
1897
200.125
96.0
time: 1.72 s (started: 2022-05-14 04:49:00 +00:00)


In [ ]:
test_pre_dup[0][0]

In [ ]:
# check #paragraph
check_out_len = []
check_bef_len = []
for idx in range(len(test_pre_dup)):
    check_bef_len.append(len(test_pre_dup[idx][0].split('\\c\\c')))
print(np.sum(check_bef_len))

for idx in range(len(check_out)):
    check_out_len.append(len(check_out[idx].split('\\c\\c')))
print(np.sum(check_out_len))

192
15084
time: 9.98 ms (started: 2022-05-14 04:49:56 +00:00)


In [ ]:
#sentences before filtered
total_len = 0
for idx in range(len(src_clusters)):
    for id in range(len(src_clusters[idx].split('\\c\\c'))):
        sents = src_clusters[idx].split('\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
print(total_len)

36696
time: 809 ms (started: 2022-06-02 18:01:01 +00:00)


In [ ]:
#sentences after filtered
total_len = 0
for idx in range(len(read_check_model)):
    for id in range(len(read_check_model[idx].split('.\\c\\c'))):
        sents = read_check_model[idx].split('.\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
print(total_len)

1853
time: 4.23 ms (started: 2022-06-02 18:01:49 +00:00)


In [ ]:
# setting for pyrouge - used below
!git clone https://github.com/bheinzerling/pyrouge.git

%cd pyrouge
!apt install libxml-parser-perl
!pip install -e .
!git clone https://github.com/andersjo/pyrouge.git rouge

%cd rouge/tools/ROUGE-1.5.5/data
!rm WordNet-2.0.exc.db
!perl ./WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db
!pyrouge_set_rouge_path /content/drive/MyDrive/MDS/GraphSum/pyrouge/rouge/tools/ROUGE-1.5.5
!chmod 777 /content/drive/MyDrive/MDS/GraphSum/pyrouge/rouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl
!python -m pyrouge.test
!pip install -U git+https://github.com/pltrdy/pyrouge
!git clone https://github.com/pltrdy/files2rouge.git     
%cd files2rouge
!python setup_rouge.py
!python setup.py install
!pip install bert-score
!pip install rouge

Cloning into 'pyrouge'...
remote: Enumerating objects: 551, done.
remote: Total 551 (delta 0), reused 0 (delta 0), pack-reused 551
Receiving objects: 100% (551/551), 123.17 KiB | 9.47 MiB/s, done.
Resolving deltas: 100% (198/198), done.
/content/pyrouge
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libauthen-sasl-perl libdata-dump-perl libencode-locale-perl
  libfile-listing-perl libfont-afm-perl libhtml-form-perl libhtml-format-perl
  libhtml-parser-perl libhtml-tagset-perl libhtml-tree-perl
  libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libio-html-perl
  libio-socket-ssl-perl liblwp-mediatypes-perl liblwp-protocol-https-perl
  libmailtools-perl libnet-http-perl libnet-smtp-ssl-perl libnet-

In [ ]:
%ls /content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered/without_update_src

bef.test.txt.src  output_intermediate.txt
time: 115 ms (started: 2022-05-21 09:48:47 +00:00)


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs3_train/filtered
from rouge import FilesRouge

files_rouge = FilesRouge()
hyp_path = 'without_update_src/bef.test.txt.src'
# hyp_path = 'aft_test_rouge_sec_txt.src'
# hyp_path = 'aft_test_rouge_txt.src'
ref_path = 'test.txt.tgt'
scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
scores

In [ ]:
# METEOR and BLEU
%cd /content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered
gen_summ = []
tgt = []
with open('gen_summ/gen_summ.test.tgt', 'r') as fread:
# with open('without_update_src/bef.test.txt.src') as fread:
# with open('aft_model_sec_txt.src', 'r') as fread:
    for line in fread.readlines():
        gen_summ.append(line.strip())
with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        tgt.append(line.strip())
assert len(gen_summ)==len(tgt)
# extract the gen_summ and tgt
gen_summ_sel = [gen_summ[idx].lstrip('{"').lstrip("{'").rstrip() for idx in range(len(gen_summ))]
tgt_sel = [tgt[idx].lstrip('{"').lstrip("{'").rstrip() for idx in range(len(tgt))]
assert len(gen_summ_sel)==len(tgt_sel)

/content/drive/MyDrive/MDS/wcep_custom/bs3_test/filtered
time: 42.8 ms (started: 2022-05-21 14:40:25 +00:00)


In [ ]:
%%capture
!pip install nltk -U

time: 5.85 s (started: 2022-05-21 14:39:36 +00:00)


In [ ]:
# calculate BLEU
from nltk.translate.bleu_score import sentence_bleu
tokens_sentences = []
candidates = []
scores_1gram, scores_2gram, scores_3gram, scores_4gram = [], [], [], []
for idx in range(len(tgt_sel)):
    ref_sentences = tgt_sel[idx].split('. ')
    tokens_sentence = []
    for sentence in ref_sentences:
        tokens_sentence.append(sentence.split())
    tokens_sentences.append(tokens_sentence)
for idx in range(len(gen_summ_sel)):
    candidate = gen_summ_sel[idx].split()
    candidates.append(candidate)
for idx in range(len(gen_summ_sel)):
    scores_1gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(1, 0, 0, 0)))
    scores_2gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(0, 1, 0, 0)))
    scores_3gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(0, 0, 1, 0)))
    scores_4gram.append(sentence_bleu(tokens_sentences[idx], candidates[idx], weights=(0, 0, 0, 1)))
print(np.mean(scores_1gram))
print(np.mean(scores_2gram))
print(np.mean(scores_3gram))
print(np.mean(scores_4gram))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

0.8327519188639616
0.6909778316198414
0.5907540467705903
0.5302029414235289
time: 12.3 s (started: 2022-05-21 14:40:28 +00:00)


In [ ]:
# calculate METEOR
import nltk
from nltk.translate.meteor_score import meteor_score as mt
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
scores = []
for idx in range(len(tgt_sel)):
    ref_sentences = tgt_sel[idx].split('. ')
    ref_tokenized = [word_tokenize(sent) for sent in ref_sentences]
    can = word_tokenize(gen_summ_sel[idx])
    scores.append(mt(ref_tokenized, can))
print(np.mean(scores))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


0.6040688693870538
time: 1min 9s (started: 2022-05-21 14:40:40 +00:00)


#### baseline 2 (wiki portal news first)

In [ ]:
src_list = []
empt_idx = []
for idx in idx_list:
    rm_id = []
    source_list = clusters[idx]['source_news_first'].split('. ')
    for id in range(len(source_list)):
        lst = source_list[id].split(' ')
        if len(lst)==1 or (lst[0].isupper() and lst[1].isupper()) or (lst[0].isdigit()):
            rm_id.append(id)
    rm_id_rev = sorted(list(set(rm_id)), reverse=True)
    for i in rm_id_rev:
        del source_list[i]
    if len(source_list)==0:
        empt_idx.append(idx)
    src_list.append(source_list)
new_idx_list = Diff(idx_list, empt_idx) # respct to original idx_list
in_idx = [id for id in range(len(src_list)) if len(src_list[id])!=0]

time: 11.1 ms (started: 2022-04-20 02:16:09 +00:00)


In [ ]:
len(new_idx_list)

337

time: 2.54 ms (started: 2022-04-20 02:16:12 +00:00)


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/baseline_2

/content/drive/MyDrive/MDS/wcep_custom/baseline_2
time: 2.12 ms (started: 2022-04-20 02:15:32 +00:00)


In [ ]:
# re-construct source
with open('test.txt.src', 'w') as fwrite:
    for idx in in_idx:
        s = '.\c\c'
        src_txt = s.join(src_list[idx])
        fwrite.write(src_txt)
        fwrite.write('\n')

time: 7.03 ms (started: 2022-04-20 02:16:16 +00:00)


In [ ]:
non_update_all_paragraph_lst = [clusters[idx]['non_update_all_paragraph'].split('\n') for idx in new_idx_list]
check_line = []
recon_line = []
with open('test.txt.src', 'r') as fread:
    for line in fread.readlines():
        check_line.append(line.strip())

time: 15.6 ms (started: 2022-04-20 02:16:17 +00:00)


In [ ]:
type(check_line[0])

str

time: 3.14 ms (started: 2022-04-20 02:16:20 +00:00)


In [ ]:
src_data_pre = []
with open('bef.test.txt.src', 'w') as fwrite:
    for idx in range(len(check_line)):
        bef_rec = []
        source_news_first = check_line[idx]
        non_update_all_paragraph = non_update_all_paragraph_lst[idx]
        bef_rec.extend(non_update_all_paragraph)
        bef_rec.append(source_news_first)
        s = '.\c\c'
        bef_rec_str = s.join(bef_rec)
        with contextlib.redirect_stdout(fwrite):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})

time: 585 ms (started: 2022-04-20 02:16:23 +00:00)


In [ ]:
import re
check_src = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        line = re.sub("[\"\'\{\}]", "", line)
        line = line.strip()
        check_src.append(line)
print(len(check_src))

with open('bef.test.txt.src', 'w') as fwrite:
    for line in check_src:
        line = line.replace('\\\\c\\\\c', '\\c\\c')
        # for without validation
        # line = line.replace('\\c\\c', ' `` ')
        fwrite.write(line+'\n')

337
time: 136 ms (started: 2022-04-20 02:16:23 +00:00)


In [ ]:
check_src_fur = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        check_src_fur.append(line)
print(len(check_src_fur))

337
time: 17.1 ms (started: 2022-04-20 02:16:27 +00:00)


In [ ]:
check_src_fur[:5]

In [ ]:
# tgt: udpate_all_paragraph
tgt_clusters = []
check_output = []

for idx in new_idx_list:
    cleaned_article = re.sub(r'\[.*?\]', "", clusters[idx]['update_first_paragraph'])
    output_article = str(cleaned_article).rstrip('\n')
    output_article = str(output_article).rstrip()
    tgt_clusters.append(output_article)

time: 3.39 ms (started: 2022-04-20 02:16:31 +00:00)


In [ ]:
with open('test.txt.tgt', 'w') as fwrite:
    for line in tgt_clusters:
        fwrite.write('%s\n' % line)

with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        check_output.append(line)

time: 419 ms (started: 2022-04-20 02:16:33 +00:00)


In [ ]:
intermediate = []

with open("output_intermediate.txt", "w") as o:
    for idx in range(len(tgt_clusters)):
        check_txt = check_output[idx]
        with contextlib.redirect_stdout(o):
            print({unicodedata.normalize('NFKD', check_txt).encode('utf-8', 'ignore').decode('utf-8')})

intermediate = []
with open('output_intermediate.txt', 'r') as fwrite:
    for line in fwrite.readlines():
        intermediate.append(line)
    for id in range(len(intermediate)):
        intermediate[id] = intermediate[id].replace("{b'", "").replace("\\n'}\n", "").replace('{b"', '').replace('\\n"}\n', '')
        re.sub("[\"\'\{\}]", "", line)
        intermediate[id] = intermediate[id].replace('.. ', '. ')

time: 445 ms (started: 2022-04-20 02:16:35 +00:00)


In [ ]:
with open('test.txt.tgt', 'w') as fwrite:
    for line in intermediate:
        fwrite.writelines(line+'\n')

time: 26.4 ms (started: 2022-04-20 02:16:36 +00:00)


In [ ]:
check_tgt_out = []
with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        check_tgt_out.append(line.strip())
print(len(check_tgt_out))

337
time: 4.68 ms (started: 2022-04-20 02:16:37 +00:00)


##### Remove duplicated sentences

In [ ]:
train_pt = torch.load('/content/drive/MyDrive/MDS_PRIMER/primer/dataset/wceptrain.pt')

time: 525 ms (started: 2022-04-20 02:16:47 +00:00)


In [ ]:
src_clusters = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        src_clusters.append(line)

time: 19.7 ms (started: 2022-04-20 02:16:52 +00:00)


In [ ]:
# create sentences for finding duplicated docs - training set
concat_sents_cluster = []
for idx in range(len(train_pt)):
    sents = []
    for id in range(len(train_pt[idx]['document'])):
        sent = train_pt[idx]['document'][id].split('\n\n')
        sents.extend(sent)
    s = '. '
    concat_sents = s.join(sents)
    concat_sents_cluster.append(concat_sents)

time: 409 ms (started: 2022-04-20 02:16:52 +00:00)


In [ ]:
#sentences before filtered
total_len = 0
total_par = 0
for idx in range(len(src_clusters)):
    for id in range(len(src_clusters[idx].split('.\\c\\c'))):
        sents = src_clusters[idx].split('.\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
        total_par+=1
print(total_len)

45569
time: 597 ms (started: 2022-04-20 02:16:53 +00:00)


In [ ]:
concat_custom = []
for idx in range(len(src_clusters)):
    pars_per = src_clusters[idx].split('.\\c\\c')
    concat_custom.append(pars_per)

time: 9.45 ms (started: 2022-04-20 02:16:55 +00:00)


In [ ]:
train_pre_dup = []
test_pre_dup = concat_custom.copy()

for idx in range(len(concat_sents_cluster)):
    ele = concat_sents_cluster[idx].split('. ')
    train_pre_dup.append(ele)

time: 707 ms (started: 2022-04-20 02:16:55 +00:00)


In [ ]:
all_train_lst = []
all_dict = {}
num_sent = 0

for id in range(len(train_pre_dup)):
    for i in range(len(train_pre_dup[id])):
        all_train_lst.append(train_pre_dup[id][i])

time: 418 ms (started: 2022-04-20 02:16:57 +00:00)


In [ ]:
aft_processed = []
for idx in range(len(test_pre_dup)):
    # splitted = [test_pre_dup[idx][id].split('. ') for id in test_pre_dup]
    processed_pars = []
    for id in range(len(test_pre_dup[idx])):
        sents_in_par = test_pre_dup[idx][id].split('. ')
        dict_idx = sents_in_par
        dict_val = [0 for _ in range(len(dict_idx))]
        dict_cus = dict(zip(dict_idx, dict_val))
        for q in all_train_lst:
            try:
                if dict_cus[q]==0: 
                    sents_in_par.remove(q)
            except:
                pass
        s = '. '
        processed_par = s.join(sents_in_par)
        processed_pars.append(processed_par)
    aft_processed.append(processed_pars)

with open('aft_test.txt.src', 'w') as fwrite:
    for idx in range(len(aft_processed)):
        src_data = aft_processed[idx]
        src_data = [x for x in src_data if x]
        s = r'.\c\c'
        src_txt = s.join(src_data)
        fwrite.write(src_txt)   

time: 1h 46min 17s (started: 2022-03-23 12:03:25 +00:00)


In [ ]:
check_out = []
with open('aft_test.txt.src', 'r') as f:
    
    for line in f.readlines():
        check_out.append(line)

time: 721 ms (started: 2022-04-20 02:17:05 +00:00)


In [ ]:
check_out[:5]

['The 2016 Atlantic hurricane season is the current tropical cyclone season in the North Atlantic Ocean, which officially began on June 1 will end on November 30. This season began nearly five months before the official start, with Hurricane Alex forming in the Northeastern Atlantic in mid-January. Tropical Storm Bonnie followed in late May, which was the first occurrence of two pre-season Atlantic storms since 2012. In June, tropical storms Colin and Danielle formed and became the earliest third and fourth named storms in the Atlantic in recorded history. In August, Hurricane Earl killed 67 people after striking Belize and Mexico..\\c\\cTo this point, most forecasting groups have expected this season to be an above average season, due to a combination of factors including an expected transition to La Niña and warmer than normal sea surface temperatures in the Gulf of Mexico, Caribbean, and Western Atlantic, despite near-average sea surface temperatures in the Atlantic Main Developmen

time: 3.89 ms (started: 2022-04-20 02:17:08 +00:00)


In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('\\c\\c', ' `` ')
    modified_out.append(line)
with open('aft_test_rouge_txt.src', 'w') as fwrite:
    for line in modified_out:
        fwrite.write(line)

time: 843 ms (started: 2022-04-20 02:17:08 +00:00)


In [ ]:
read_check = []
with open('aft_test_rouge_txt.src', 'r') as fread:
    for line in fread.readlines():
        read_check.append(line)
print(len(read_check))

337
time: 19.8 ms (started: 2022-04-20 02:17:12 +00:00)


In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('.\\c\\c', '\c\c')
    modified_out.append(line)
with open('aft_test_model_txt.src', 'w') as fwrite:
    for line in modified_out:
        line_encoded = unicodedata.normalize('NFKD', line).encode('utf-8', 'ignore').decode('utf-8')
        # line_encoded = line.encode('unicode_escape')
        fwrite.write(str(line_encoded))

read_check_model = []
with open('aft_test_model_txt.src', 'r') as fread:
    for line in fread.readlines():
        read_check_model.append(line.replace('.\\c\\c', '\c\c'))
print(len(read_check_model))

337
time: 748 ms (started: 2022-04-20 02:17:12 +00:00)


In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('.\\c\\c', '. ')
    modified_out.append(line)
with open('aft_test_rouge_period_txt.src', 'w') as fwrite:
    for line in modified_out:
        line_encoded = unicodedata.normalize('NFKD', line).encode('utf-8', 'ignore').decode('utf-8')
        # line_encoded = line.encode('unicode_escape')
        fwrite.write(str(line_encoded))

read_check_model = []
with open('aft_test_rouge_period_txt.src', 'r') as fread:
    for line in fread.readlines():
        read_check_model.append(line.replace('.\\c\\c', '. '))
print(len(read_check_model))

337
time: 731 ms (started: 2022-04-20 02:17:13 +00:00)


In [ ]:
cnt_par = []
for idx in range(len(check_out)):
    cnt_single_par = len(check_out[idx].split('.\\c\\c'))
    cnt_par.append(cnt_single_par)
print('#paragraphs in cluster')
print(np.min(cnt_par))
print(np.max(cnt_par))
print(np.mean(cnt_par))
print(np.median(cnt_par))

cnt_sents = []
for idx in range(len(check_out)):
    cnt_single = 0
    for id in range(len(check_out[idx].split('.\\c\\c'))):
        single_par = check_out[idx].split('.\\c\\c')[id]
        cnt_single+=len(single_par.split('. '))
    cnt_sents.append(cnt_single)
print('#sentences in paragraph')
print(np.min(cnt_sents))
print(np.max(cnt_sents))
print(np.mean(cnt_sents))
print(np.median(cnt_sents))

cnt_single_sent = []
for idx in range(len(check_out)):
    for id in range(len(check_out[idx].split('\\c\\c'))):
        single_par = check_out[idx].split('.\\c\\c')[id]
        cnt_sent_in_par = len(single_par.split('. '))
        cnt_single_sent.append(cnt_sent_in_par)


#paragraphs in cluster
2
370
40.26112759643917
24.0
#sentences in paragraph
3
1045
132.49554896142433
68.0
time: 1.17 s (started: 2022-04-20 02:18:02 +00:00)


In [ ]:
# check #paragraph
check_out_len = []
check_bef_len = []
for idx in range(len(test_pre_dup)):
    check_bef_len.append(len(test_pre_dup[idx]))
print(np.sum(check_bef_len))

for idx in range(len(check_out)):
    check_out_len.append(len(check_out[idx].split('.\\c\\c')))
print(np.sum(check_out_len))

13665
13568
time: 10.8 ms (started: 2022-04-20 02:19:26 +00:00)


In [ ]:
#sentences before filtered
total_len = 0
for idx in range(len(src_clusters)):
    for id in range(len(src_clusters[idx].split('\\c\\c'))):
        sents = src_clusters[idx].split('\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
print(total_len)

45569
time: 531 ms (started: 2022-04-20 02:19:49 +00:00)


In [ ]:
#sentences after filtered
total_len = 0
for idx in range(len(check_out)):
    for id in range(len(check_out[idx].split('.\\c\\c'))):
        sents = check_out[idx].split('.\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
print(total_len)

44651
time: 584 ms (started: 2022-04-20 02:19:50 +00:00)


#### baseline 3 (wiki portal news full)

In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom
import json
clusters = []
with open("new_title_page_dataset_nextline.txt", 'r') as fread:
    for line in fread.readlines():
        clusters.append(json.loads(line.strip()))

/content/drive/MyDrive/MDS/wcep_custom
time: 2.62 s (started: 2022-03-23 14:48:22 +00:00)


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/baseline_3

/content/drive/MyDrive/MDS/wcep_custom/baseline_3
time: 3.21 ms (started: 2022-03-24 04:11:21 +00:00)


In [ ]:
src_list = []
empt_idx = []
for idx in idx_list:
    rm_id = []
    source_list = clusters[idx]['source_news_full'].split('. ')
    for id in range(len(source_list)):
        lst = source_list[id].split(' ')
        if len(lst)<5 or (lst[0].isupper() and lst[1].isupper()) or (lst[0].isdigit()):
            rm_id.append(id)
    rm_id_rev = sorted(list(set(rm_id)), reverse=True)
    for i in rm_id_rev:
        del source_list[i]
    if len(source_list)==0:
        empt_idx.append(idx)
    src_list.append(source_list)
new_idx_list = Diff(idx_list, empt_idx) # respct to original idx_list - for tgt selection
in_idx = [id for id in range(len(src_list)) if len(src_list[id])!=0]
for idx in range(len(src_list)):
    for id in range(len(src_list[idx])):
        src_list[idx][id] = src_list[idx][id].replace('\n', ' ')

time: 39.7 ms (started: 2022-03-24 04:30:20 +00:00)


In [ ]:
# re-construct source
with open('test.txt.src', 'w') as fwrite:
    for idx in in_idx:
        s = '.\c\c'
        src_txt = s.join(src_list[idx])
        # src_txt = src_txt.replace('\n', ' ')
        fwrite.write(src_txt.strip())
        fwrite.write('\n')

time: 396 ms (started: 2022-03-24 04:30:31 +00:00)


In [ ]:
non_update_all_paragraph_lst = [clusters[idx]['non_update_all_paragraph'].split('\n') for idx in new_idx_list]
check_line = []
with open('test.txt.src', 'r') as fread:
    for line in fread.readlines():
        check_line.append(line.strip())

time: 23.4 ms (started: 2022-03-24 04:30:34 +00:00)


In [ ]:
check_line[:5]

['The post-tropical cyclone is moving toward the south near 6 mph (9 km/h).\\c\\cA turn toward the southwest and west is expected over the weekend, followed by a slow motion toward the northwest Sunday night and Monday.\\c\\cMaximum sustained winds are near 30 mph (45 km/h) with higher gusts.\\c\\cGradual weakening is anticipated and Chantal is forecast to dissipate on Monday.\\c\\cThe estimated minimum central pressure is 1014 mb (29.95 inches).\\c\\cAdditional information on this system can be found in High Seas Forecasts issued by the National Weather Service, under AWIPS header NFDHSFAT1, WMO header FZNT01 KWBC, and online at ocean.weather.gov/shtml/NFDHSFAT1.php $$ Forecaster Zelinsky',
 '(CNN) Brazil\'s first female president is out of a job, but not barred from the ballot if she wants to run again. The South American country\'s Senate voted 61-20 Wednesday to remove President Dilma Rousseff from office, finding her guilty of breaking budgetary laws in an impeachment trial. Temer

time: 2.96 ms (started: 2022-03-24 04:30:36 +00:00)


In [ ]:
src_data_pre = []
with open('bef.test.txt.src', 'w') as fwrite:
    for idx in range(len(check_line)):
        bef_rec = []
        source_news_full = check_line[idx]
        non_update_all_paragraph = non_update_all_paragraph_lst[idx]
        bef_rec.extend(non_update_all_paragraph)
        bef_rec.extend(source_news_full.split('.\\c\\c'))
        s = '.\c\c'
        bef_rec_str = s.join(bef_rec)
        with contextlib.redirect_stdout(fwrite):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})

time: 684 ms (started: 2022-03-24 04:30:40 +00:00)


In [ ]:
import re
check_src = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        line = re.sub("[\"\'\{\}]", "", line)
        line = line.replace('.\\\\c\\\\c', '.\\c\\c')
        line = line.strip()
        check_src.append(line)
print(len(check_src))

with open('bef.test.txt.src', 'w') as fwrite:
    for line in check_src:
        fwrite.write(line+'\n')

361
time: 144 ms (started: 2022-03-24 04:30:41 +00:00)


In [ ]:
check_src_fur = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        check_src_fur.append(line)
print(len(check_src_fur))

361
time: 19.6 ms (started: 2022-03-24 04:30:49 +00:00)


In [ ]:
check_src_fur[:5]

['The 2016 Atlantic hurricane season is the current tropical cyclone season in the North Atlantic Ocean, which officially began on June 1 will end on November 30. This season began nearly five months before the official start, with Hurricane Alex forming in the Northeastern Atlantic in mid-January. Tropical Storm Bonnie followed in late May, which was the first occurrence of two pre-season Atlantic storms since 2012. In June, tropical storms Colin and Danielle formed and became the earliest third and fourth named storms in the Atlantic in recorded history. In August, Hurricane Earl killed 67 people after striking Belize and Mexico..\\c\\cTo this point, most forecasting groups have expected this season to be an above average season, due to a combination of factors including an expected transition to La Niña and warmer than normal sea surface temperatures in the Gulf of Mexico, Caribbean, and Western Atlantic, despite near-average sea surface temperatures in the Atlantic Main Developmen

time: 9.49 ms (started: 2022-03-23 14:55:52 +00:00)


In [ ]:
# tgt: udpate_all_paragraph
tgt_clusters = []
check_output = []

for idx in new_idx_list:
    cleaned_article = re.sub(r'\[.*?\]', "", clusters[idx]['update_first_paragraph'])
    output_article = str(cleaned_article).rstrip('\n')
    output_article = str(output_article).rstrip()
    tgt_clusters.append(output_article)

time: 4.15 ms (started: 2022-03-24 04:31:12 +00:00)


In [ ]:
with open('test.txt.tgt', 'w') as fwrite:
    for line in tgt_clusters:
        fwrite.write('%s\n' % line)

with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        check_output.append(line)

time: 440 ms (started: 2022-03-24 04:31:12 +00:00)


In [ ]:
intermediate = []

with open("output_intermediate.txt", "w") as o:
    for idx in range(len(tgt_clusters)):
        check_txt = check_output[idx]
        with contextlib.redirect_stdout(o):
            print({unicodedata.normalize('NFKD', check_txt).encode('utf-8', 'ignore').decode('utf-8')})

intermediate = []
with open('output_intermediate.txt', 'r') as fwrite:
    for line in fwrite.readlines():
        intermediate.append(line)
    for id in range(len(intermediate)):
        intermediate[id] = intermediate[id].replace("{b'", "").replace("\\n'}\n", "").replace('{b"', '').replace('\\n"}\n', '')
        re.sub("[\"\'\{\}]", "", line)
        intermediate[id] = intermediate[id].replace('.. ', '. ')

time: 407 ms (started: 2022-03-24 04:31:12 +00:00)


In [ ]:
with open('test.txt.tgt', 'w') as fwrite:
    for line in intermediate:
        fwrite.writelines(line+'\n')

time: 28.3 ms (started: 2022-03-24 04:31:13 +00:00)


In [ ]:
check_tgt_out = []
with open('test.txt.tgt', 'r') as fread:
    for line in fread.readlines():
        check_tgt_out.append(line.strip())
print(len(check_tgt_out))

361
time: 3.94 ms (started: 2022-03-24 04:31:14 +00:00)


##### Remove duplicated sentences

In [ ]:
train_pt = torch.load('/content/drive/MyDrive/MDS_PRIMER/primer/dataset/train.pt')

time: 4.48 s (started: 2022-03-24 04:31:17 +00:00)


In [ ]:
src_clusters = []
with open('bef.test.txt.src', 'r') as fread:
    for line in fread.readlines():
        src_clusters.append(line)

time: 21.1 ms (started: 2022-03-24 04:31:21 +00:00)


In [ ]:
src_clusters[:5]

['The 2016 Atlantic hurricane season is the current tropical cyclone season in the North Atlantic Ocean, which officially began on June 1 will end on November 30. This season began nearly five months before the official start, with Hurricane Alex forming in the Northeastern Atlantic in mid-January. Tropical Storm Bonnie followed in late May, which was the first occurrence of two pre-season Atlantic storms since 2012. In June, tropical storms Colin and Danielle formed and became the earliest third and fourth named storms in the Atlantic in recorded history. In August, Hurricane Earl killed 67 people after striking Belize and Mexico..\\c\\cTo this point, most forecasting groups have expected this season to be an above average season, due to a combination of factors including an expected transition to La Niña and warmer than normal sea surface temperatures in the Gulf of Mexico, Caribbean, and Western Atlantic, despite near-average sea surface temperatures in the Atlantic Main Developmen

time: 3.78 ms (started: 2022-03-24 04:31:21 +00:00)


In [ ]:
# create sentences for finding duplicated docs - training set
concat_sents_cluster = []
for idx in range(len(train_pt)):
    sents = []
    for id in range(len(train_pt[idx]['document'])):
        sent = train_pt[idx]['document'][id].split('\n\n')
        sents.extend(sent)
    s = '. '
    concat_sents = s.join(sents)
    concat_sents_cluster.append(concat_sents)

time: 396 ms (started: 2022-03-24 04:31:22 +00:00)


In [ ]:
#sentences before filtered
total_len = 0
total_par = 0
for idx in range(len(src_clusters)):
    for id in range(len(src_clusters[idx].split('.\\c\\c'))):
        sents = src_clusters[idx].split('.\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
        total_par+=1
print(total_len)

58192
time: 814 ms (started: 2022-03-24 04:31:25 +00:00)


In [ ]:
concat_custom = []
for idx in range(len(src_clusters)):
    pars_per = src_clusters[idx].split('.\\c\\c')
    concat_custom.append(pars_per)

time: 15.9 ms (started: 2022-03-24 04:32:04 +00:00)


In [ ]:
train_pre_dup = []
test_pre_dup = concat_custom.copy()

for idx in range(len(concat_sents_cluster)):
    ele = concat_sents_cluster[idx].split('. ')
    train_pre_dup.append(ele)

time: 830 ms (started: 2022-03-24 04:32:05 +00:00)


In [ ]:
all_train_lst = []
all_dict = {}
num_sent = 0

for id in range(len(train_pre_dup)):
    for i in range(len(train_pre_dup[id])):
        all_train_lst.append(train_pre_dup[id][i])

time: 384 ms (started: 2022-03-24 04:32:08 +00:00)


In [ ]:
aft_processed = []
for idx in range(len(test_pre_dup)):
    # splitted = [test_pre_dup[idx][id].split('. ') for id in test_pre_dup]
    processed_pars = []
    for id in range(len(test_pre_dup[idx])):
        sents_in_par = test_pre_dup[idx][id].split('. ')
        dict_idx = sents_in_par
        dict_val = [0 for _ in range(len(dict_idx))]
        dict_cus = dict(zip(dict_idx, dict_val))
        for q in all_train_lst:
            try:
                if dict_cus[q]==0: 
                    sents_in_par.remove(q)
            except:
                pass
        s = '. '
        processed_par = s.join(sents_in_par)
        processed_pars.append(processed_par)
    aft_processed.append(processed_pars)

with open('aft_test.txt.src', 'w') as fwrite:
    for idx in range(len(aft_processed)):
        src_data = aft_processed[idx]
        src_data = [x for x in src_data if x]
        s = r'.\c\c'
        src_txt = s.join(src_data)
        fwrite.write(src_txt)

time: 2h 21min 7s (started: 2022-03-24 04:32:09 +00:00)


In [ ]:
check_out = []
with open('aft_test.txt.src', 'r') as f:
    for line in f.readlines():
        check_out.append(line)

time: 22.6 ms (started: 2022-03-24 07:04:45 +00:00)


In [ ]:
check_out[:5]

In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('\\c\\c', ' `` ')
    modified_out.append(line)
with open('aft_test_rouge_txt.src', 'w') as fwrite:
    for line in modified_out:
        fwrite.write(line)

time: 531 ms (started: 2022-03-24 07:04:51 +00:00)


In [ ]:
read_check = []
with open('aft_test_rouge_txt.src', 'r') as fread:
    for line in fread.readlines():
        read_check.append(line)
print(len(read_check))

361
time: 26.5 ms (started: 2022-03-24 07:04:53 +00:00)


In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('.\\c\\c', '\c\c')
    modified_out.append(line)
with open('aft_test_model_txt.src', 'w') as fwrite:
    for line in modified_out:
        line_encoded = unicodedata.normalize('NFKD', line).encode('utf-8', 'ignore').decode('utf-8')
        # line_encoded = line.encode('unicode_escape')
        fwrite.write(str(line_encoded))

read_check_model = []
with open('aft_test_model_txt.src', 'r') as fread:
    for line in fread.readlines():
        read_check_model.append(line.replace('.\\c\\c', '\c\c'))
print(len(read_check_model))

361
time: 615 ms (started: 2022-03-24 07:04:56 +00:00)


In [ ]:
# re-construct src for model validation or not
modified_out = []
for idx in range(len(check_out)):
    line = check_out[idx].replace('.\\c\\c', '. ')
    modified_out.append(line)
with open('aft_test_rouge_period_txt.src', 'w') as fwrite:
    for line in modified_out:
        line_encoded = unicodedata.normalize('NFKD', line).encode('utf-8', 'ignore').decode('utf-8')
        # line_encoded = line.encode('unicode_escape')
        fwrite.write(str(line_encoded))

read_check_model = []
with open('aft_test_rouge_period_txt.src', 'r') as fread:
    for line in fread.readlines():
        read_check_model.append(line.replace('.\\c\\c', '. '))
print(len(read_check_model))

361
time: 597 ms (started: 2022-03-24 07:04:58 +00:00)


In [ ]:
cnt_par = []
for idx in range(len(check_out)):
    cnt_single_par = len(check_out[idx].split('.\\c\\c'))
    cnt_par.append(cnt_single_par)

print(np.mean(cnt_par))
print(np.max(cnt_par))
print(np.min(cnt_par))
print(np.median(cnt_par))

cnt_sents = []
for idx in range(len(check_out)):
    cnt_single = 0
    for id in range(len(check_out[idx].split('.\\c\\c'))):
        single_par = check_out[idx].split('.\\c\\c')[id]
        cnt_single+=len(single_par.split('. '))
    cnt_sents.append(cnt_single)
print(np.mean(cnt_sents))
print(np.max(cnt_sents))
print(np.min(cnt_sents))
print(np.median(cnt_sents))

cnt_single_sent = []
for idx in range(len(check_out)):
    for id in range(len(check_out[idx].split('\\c\\c'))):
        single_par = check_out[idx].split('.\\c\\c')[id]
        cnt_sent_in_par = len(single_par.split('. '))
        cnt_single_sent.append(cnt_sent_in_par)

48.88642659279778
373
2
33.0
154.2409972299169
1056
7
97.0
time: 1.53 s (started: 2022-03-24 07:04:59 +00:00)


In [ ]:
# check #paragraph
check_out_len = []
check_bef_len = []
for idx in range(len(test_pre_dup)):
    check_bef_len.append(len(test_pre_dup[idx]))
print(np.sum(check_bef_len))

for idx in range(len(check_out)):
    check_out_len.append(len(check_out[idx].split('.\\c\\c')))
print(np.sum(check_out_len))

18505
17648
time: 13.8 ms (started: 2022-03-24 07:05:01 +00:00)


In [ ]:
#sentences before filtered
total_len = 0
for idx in range(len(src_clusters)):
    for id in range(len(src_clusters[idx].split('\\c\\c'))):
        sents = src_clusters[idx].split('\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
print(total_len)

58192
time: 735 ms (started: 2022-03-24 07:05:03 +00:00)


In [ ]:
#sentences after filtered
total_len = 0
for idx in range(len(check_out)):
    for id in range(len(check_out[idx].split('.\\c\\c'))):
        sents = check_out[idx].split('.\\c\\c')
        per_len = len(sents[id].split('. '))
        total_len+=per_len
print(total_len)

55681
time: 772 ms (started: 2022-03-24 07:05:06 +00:00)


#### Further: Splitting the training data

In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs1_train

/content/drive/MyDrive/MDS/wcep_custom/bs1_train
time: 1.9 ms (started: 2022-04-25 08:24:02 +00:00)


In [ ]:
train_pt = torch.load('wceptrain.pt')
# train_df = pd.read_csv('train_df.csv')

time: 100 ms (started: 2022-04-25 08:24:03 +00:00)


In [ ]:
len(train_pt)

1808

time: 2.68 ms (started: 2022-04-25 08:24:04 +00:00)


In [ ]:
type(train_pt[0]['document'])

list

time: 4.52 ms (started: 2022-04-25 06:40:19 +00:00)


In [ ]:
train_pt[0].keys()

dict_keys(['document', 'summary'])

time: 2.66 ms (started: 2022-04-25 06:42:14 +00:00)


##### Further - exp.2

In [ ]:
# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

time: 2.59 ms (started: 2022-04-25 08:24:22 +00:00)


In [ ]:
%cd /content/drive/MyDrive/MDS/wcep_custom/bs1_train
train_pt = torch.load('wceptrain.pt')

/content/drive/MyDrive/MDS/wcep_custom/bs1_train
time: 110 ms (started: 2022-04-25 08:24:34 +00:00)


In [ ]:
# filter by language of docs==English
'''
wrap = []
for idx in range(len(train_pt)):
    idx_content = {}
    doc_string = '. '.join([single for single in train_pt[idx]['document']])
    if isEnglish(doc_string):
        idx_content['document'] = train_pt[idx]['document']
        idx_content['summary'] = train_pt[idx]['summary']
        wrap.append(idx_content)
torch.save(wrap, 'train_splitted/wceptrain_en_non.pt')
'''

time: 92.1 ms (started: 2022-04-21 15:11:17 +00:00)


In [ ]:
# filter by language of summ==English
wrap = []
for idx in range(len(train_pt)):
    idx_content = {}
    summ_str = train_pt[idx]['summary']
    if isEnglish(summ_str):
        idx_content['document'] = train_pt[idx]['document']
        idx_content['summary'] = train_pt[idx]['summary']
        wrap.append(idx_content)
torch.save(wrap, 'train_splitted/wceptrain_en.pt')

time: 668 ms (started: 2022-04-25 08:25:27 +00:00)


In [ ]:
wceptrain_en_pt = torch.load('train_splitted/wceptrain_en.pt')
len(wceptrain_en_pt)

1209

time: 68.4 ms (started: 2022-04-25 08:35:09 +00:00)


In [ ]:
wceptrain_en_pt[0]['document']

list

time: 3.89 ms (started: 2022-04-25 08:30:57 +00:00)


In [ ]:
%pwd

'/content/drive/MyDrive/MDS/wcep_custom/bs1_train'

time: 3.07 ms (started: 2022-04-25 08:33:46 +00:00)


In [ ]:
###
wrap = []
for idx in range(len(wceptrain_en_pt)):
    wrap_lst = []
    for id in range(len(wceptrain_en_pt[idx]['document'])):
        s = ' '
        filtered_str = s.join(wceptrain_en_pt[idx]['document'][id].split()[:1200])
        wrap_lst.append(filtered_str)
    idx_content = {}
    idx_content['document'] = wrap_lst
    idx_content['summary'] = wceptrain_en_pt[idx]['summary']
    wrap.append(idx_content)
torch.save(wrap, 'wceptrain_en_trunc.pt')

time: 505 ms (started: 2022-04-25 08:34:31 +00:00)


In [ ]:
wceptrain_en_trunc_pt = torch.load('wceptrain_en_trunc.pt')
len(wceptrain_en_trunc_pt)

1209

time: 66.8 ms (started: 2022-04-25 08:35:21 +00:00)


In [ ]:
# extract the first 100 english clusters
wrap = []
for idx in range(0, 100):
    idx_content = {}
    idx_content['document'] = wceptrain_en[idx]['document']
    idx_content['summary'] = wceptrain_en[idx]['summary']
    wrap.append(idx_content)
torch.save(wrap, 'train_splitted/wceptrain_en_0_100.pt')

time: 18.3 ms (started: 2022-04-21 15:00:50 +00:00)


In [ ]:
wceptrain_en_0_100[0].keys()

dict_keys(['document', 'summary'])

time: 3.74 ms (started: 2022-04-21 15:01:27 +00:00)


In [ ]:
# extract the specific range of english clusters
wrap = []
for idx in range(150, 160):
    idx_content = {}
    idx_content['document'] = wceptrain_en_non[idx]['document']
    idx_content['summary'] = wceptrain_en_non[idx]['summary']
    wrap.append(idx_content)
torch.save(wrap, 'train_splitted/wceptrain_en_non_150_160.pt')

time: 7.03 ms (started: 2022-04-23 12:33:17 +00:00)


In [ ]:
# extract the first 100 english clusters
wrap = []
idx = 152
idx_content = {}
idx_content['document'] = wceptrain_en_non[idx]['document']
idx_content['summary'] = wceptrain_en_non[idx]['summary']
wrap.append(idx_content)
torch.save(wrap, 'train_splitted/wceptrain_en_non_152.pt')

time: 7.12 ms (started: 2022-04-23 14:40:55 +00:00)


In [ ]:
# idx = 0
# idx_content = {}
# idx_content['document'] = wcep_test[idx]['document']
# idx_content['summary'] = wcep_test[idx]['summary']
# wrap.append(idx_content)
# torch.save(wrap, 'train_splitted/wceptest_000.pt')

time: 7.64 ms (started: 2022-04-21 17:15:32 +00:00)


#### Check if the sentences from training set and testing set are repeated

In [ ]:
%cd /content/drive/MyDrive/MDS_PRIMER/primer/dataset
old_wceptrain = torch.load('/content/drive/MyDrive/MDS_PRIMER/primer/dataset/wcep-10/wceptrain.pt')
wceptrain_pt = torch.load('wceptrain.pt')
# wceptest_pt = torch.load('wceptest.pt')
wceptest_pt = torch.load('/content/drive/MyDrive/MDS/wcep_custom/bs1_test/wceptest_checktraintest.pt')
wcepvalid_pt = torch.load('/content/drive/MyDrive/MDS/wcep_custom/bs1_valid/wceptest_checktrainvalid.pt')
summ_clusters = []
%cd /content/drive/MyDrive/MDS/wcep_custom/bs1_valid
with open("test.txt.tgt", 'r', encoding="utf-8") as fread:
    for line in fread.readlines():
        summ_clusters.append(line)

/content/drive/MyDrive/MDS_PRIMER/primer/dataset
/content/drive/MyDrive/MDS/wcep_custom/bs1_valid
time: 1.76 s (started: 2022-04-27 05:21:36 +00:00)


In [ ]:
type(wceptrain_pt[0]['document'])

list

time: 3.63 ms (started: 2022-04-27 05:11:58 +00:00)


In [ ]:
sents_oldtrain = []
sents_test = []
sents_train = []
sents_valid = []

for idx in range(len(old_wceptrain)):
    for id in range(len(old_wceptrain[idx]['document'])):
        for i in range(len(old_wceptrain[idx]['document'][id].split('\n\n'))):
            par = old_wceptrain[idx]['document'][id].split('\n\n')[i]
            sents_oldtrain.extend(par.split('. '))

for idx in range(len(wceptrain_pt)):
    for id in range(len(wceptrain_pt[idx]['document'])):
        for i in range(len(wceptrain_pt[idx]['document'][id].split('\n\n'))):
            par = wceptrain_pt[idx]['document'][id].split('\n\n')[i]
            sents_train.extend(par.split('. '))

for idx in range(len(wcepvalid_pt)):
    for id in range(len(wcepvalid_pt[idx]['document'])):
        for i in range(len(wcepvalid_pt[idx]['document'][id].split('\n\n'))):
            par = wcepvalid_pt[idx]['document'][id].split('\n\n')[i]
            sents_valid.extend(par.split('. '))

for idx in range(len(wceptest_pt)):
    for id in range(len(wceptest_pt[idx]['document'])):
        for i in range(len(wceptest_pt[idx]['document'][id].split('\n\n'))):
            par = wceptest_pt[idx]['document'][id].split('\n\n')[i]
            sents_test.extend(par.split('. '))
print(len(sents_oldtrain))
print(len(sents_train))
print(len(sents_valid))
print(len(sents_test))

1501554
173829
37674
18396
time: 5.84 s (started: 2022-04-27 05:22:43 +00:00)


In [ ]:
num=0
for sent in sents_test:
    if sent in sents_train:
        num+=1
print(num)

44
time: 1min 21s (started: 2022-04-27 04:29:59 +00:00)


In [ ]:
num=0
for sent in sents_test:
    if sent in sents_valid:
        num+=1
print(num)

18
time: 13.6 s (started: 2022-04-27 04:32:02 +00:00)


In [ ]:
num=0
for sent in sents_valid:
    if sent in sents_train:
        num+=1
print(num)

784
time: 3min 11s (started: 2022-04-27 05:13:14 +00:00)


In [ ]:
num=0
for sent in sents_train:
    if sent in sents_oldtrain:
        num+=1
print(num)

In [ ]:
# [] -> [article] -> [paragraph] -> [sentence] -> [paragraph] -> [article]